# Lab Tutorial 04
### Learning Outcomes 
In this tutorial we will review and practice what we have learned so far and learn new stuff to be able to write a function for permutation testing + calculating p-values for hypothesis testing. <br>
We will be using StudentsPerformance dataset from the second lab tutorial.<br>
Again the dataset can be found in:
https://www.kaggle.com/spscientist/students-performance-in-exams


# Importing packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Load in the dataset

In [3]:
df = pd.read_csv("StudentsPerformance.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## A contingency table again!
_Question 1 - Create contingency table of ethnicity by gender

In [7]:
ctab = pd.crosstab(df['race/ethnicity'], df['gender'], margins=True)
display(ctab)

gender,female,male,All
race/ethnicity,,,
group A,36,53,89
group B,104,86,190
group C,180,139,319
group D,129,133,262
group E,69,71,140
All,518,482,1000


In [9]:
p_groupA = ctab['All']['group A']/ctab['All']['All']
p_groupA

np.float64(0.089)

Separating two groups in terms of their race: A not A

In [13]:
ctab2 = pd.crosstab(df['race/ethnicity'] == "group A", df['gender'], margins=True)
ctab2

gender,female,male,All
race/ethnicity,,,
False,482,429,911
True,36,53,89
All,518,482,1000


In [16]:
p_groupA = ctab2['All'][True]/ctab2['All']['All']
p_groupA


np.float64(0.089)

In [17]:
p_groupnotA = ctab2['All'][False]/ctab2['All']['All']
p_groupnotA

np.float64(0.911)

# Defining functions again!

## Write a function that takes in a dataframe and a column name and returns the mean and std of that column

In [18]:
def my_function(data, colname):
    mean_col = data[colname].mean()
    std_col = data[colname].std()

    return (mean_col, std_col)

### Use the function to get the mean and std of column "math score"

In [19]:
[m, s] = my_function(df, 'math score')
print(f"the mean is {m}")
print(f"the std is {s}")

the mean is 66.089
the std is 15.16308009600945


## Write a function that takes in a dataframe and calculate the probability of all the races
Useful when you are doing this question _(What we are doing here is more complicated than what you need to do for the question)_ :<br>
Task 1/Question 3 - Define a function that calculates this difference in probability (10pts)

In [33]:
def prob_race(data, nvar = 'race/ethnicity'):
    races = data.groupby(nvar).groups.keys()

    probs = np.zeros((1, len(races)))

    i = 0
    for r in races:
        ctab_tmp = pd.crosstab(df[nvar] == r, df['gender'], margins=True)
        prob_tmp = ctab_tmp['All'][True]/ctab_tmp['All']['All']
        probs[0, i] = prob_tmp
        i += 1

    return probs


In [38]:
P = prob_race(df)
P

array([[0.089, 0.19 , 0.319, 0.262, 0.14 ]])

### Use the funciton to calculate the probabilities

In [39]:
prob_race(df, 'gender')

array([[0.518, 0.482]])

## Write a function that takes the number of rows and columns of a numpy array and returns a numpy array with elements drawn from normal distribution (a random 2D numpy array)
This is just for practicing function. It is not usually the way we use to create a matrix of random numbers <br>
Use __np.random.normal__ : https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html

In [42]:
def rand_array(nrow = 2, ncol = 2):
    out_nump = np.zeros((nrow, ncol))

    for i in np.arange(nrow):
        for j in np.arange(ncol):
            out_nump[i,j] = np.random.normal(0,1) # 0 is the mean and 1 is the std
    
    return out_nump


In [41]:
np.arange(5)

array([0, 1, 2, 3, 4])

### Use the function to create a 3 by 3 array of random numbers

In [53]:
A = rand_array(3,3)

### Print all the entries (with 3 decimal points) with row and col number:
the output should be like:<br>
"The entry on row 1 col 2 is 0.1"

In [56]:
[r, c] = A.shape

for i in np.arange(r):
    for j in np.arange(c):
        print(f"The entry cell on row %d and colum %d is %.3f" %(i, j, A[i,j]))

The entry cell on row 0 and colum 0 is -1.080
The entry cell on row 0 and colum 1 is -0.314
The entry cell on row 0 and colum 2 is 1.291
The entry cell on row 1 and colum 0 is 0.167
The entry cell on row 1 and colum 1 is -0.110
The entry cell on row 1 and colum 2 is 0.631
The entry cell on row 2 and colum 0 is 0.482
The entry cell on row 2 and colum 1 is 0.111
The entry cell on row 2 and colum 2 is 0.893


### Practice more:
https://www.w3resource.com/python-exercises/python-functions-exercises.php

# Problem Description
An increasingly common statistical tool for determining whether a difference is significant is the randomization or permutation test. A randomization test builds - rather than assumes – a distribution of the test statistics. This is achieved by exchanging or permuting variables which are “exchangeable” under the Null-hypothesis. A randomization test gives a simple way to compute the distribution for any test statistic under the null hypothesis that the dependent variable is statistically not dependent of the shuffled variable.<br><br><br>
Assuming that the gender variable is exchangeable, randomly shuffle the dataframe.<br>
The docs that you will need:
* https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.shuffle.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html


We want to test the hypothesis that male students are more likely than female students to take test preparation course. Under this hypothesis the p(test|male) should be higher than p(test|female).

## 1. create a copy of the dataframe
you don't want to change the original dataframe cause in each iteration, you want to shuffle the original one.

In [65]:
df_copy = df.copy()
df_copy.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## 2. create an array that represent the indices
as the indices are integers, you can create an array, having the same numbers as the indices in the dataframe and use it as the index!

In [78]:
inds = np.arange(1000)
inds

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

## 3. use np.random.shuffle to randomly shuffle indices.
the array that represents the indices will be shuffled.

In [79]:
np.random.shuffle(inds)
print(inds)

[569 658 286 609  98 375 689 792 210 874 224 841 748 137 504 758 324 417
 320 469  52 766 883 154 701 746 636 513 978 141 483 915 413 587 801 237
  41 404 643 975 401 257 855 142 246 940 785 558 783 967 140 887 768 853
 111 303 707  48  40 869 962  57 654 510 221  97  62 298 812 449 903 244
 183 156 663 825 366 329 642 332 179 791 149 361 845 143 214 379 674 633
 101 651 794 780 842 150 590 606 705 993 564 703 828 282 751 374 833 457
 133 909 895 817 334 615 555 509 586   4 943 549 195 387 835 809 408 613
 388 852 382 157  46  85 611 650  21 697 370 458 559 818 146 238 882 204
 676 728 868 155 717 931 715 944  89 682 745 422 169 583 253 202 541 597
 519 398  74 891 346 772 311 393 953 738 295 885 600 344 545 846 108 635
 899 608 938 679 740 511 556 534  29 423 369 313 972  53 207 675  50 959
 380 245  59 546 949 242  14 436 942 849 984 348 259 836 435 151 129 358
 276 297 291 184 240   8 958 974 770   0 261 284 832 409 110 399 983 218
 339 496 460 479 798 316 349 439 360 872 727 234 28

## 4. shuffle the exchangeable variable
Which variable is exchangeable under the null hypothesis?

In [80]:
tmp = df['gender'].iloc[inds]
tmp

569      male
658    female
286      male
609    female
98     female
        ...  
10       male
134      male
795    female
275      male
581    female
Name: gender, Length: 1000, dtype: object

## 5. reorder the shuffled indices. 
You will be putting the shuffled variable back to the dataframe. You want the indices for your dataframe to start from 0 (like the original one). You don't want to use the shuffled indices.<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.reset_index.html

In [81]:
tmp = tmp.reset_index(drop = True)
tmp

0        male
1      female
2        male
3      female
4      female
        ...  
995      male
996      male
997    female
998      male
999    female
Name: gender, Length: 1000, dtype: object

## 6. put the shuffled exchangeable variable back into the copy of the dataframe

In [82]:
df_copy['gender'] = tmp
df_copy.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,male,group B,master's degree,standard,none,90,95,93
3,female,group A,associate's degree,free/reduced,none,47,57,44
4,female,group C,some college,standard,none,76,78,75


## Now compare the original dataframe with the copy you created!

In [83]:
display(df.head())
display(df_copy.head())

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,male,group B,master's degree,standard,none,90,95,93
3,female,group A,associate's degree,free/reduced,none,47,57,44
4,female,group C,some college,standard,none,76,78,75


## Check to see if the proportion of each category of the exchangeable variable changed

In [84]:
cTab3 = pd.crosstab(df['test preparation course'], df['gender'], margins = True)
cTab4 = pd.crosstab(df_copy['test preparation course'], df_copy['gender'], margins = True)
display(cTab3)
display(cTab4)

gender,female,male,All
test preparation course,,,
completed,184,174,358
none,334,308,642
All,518,482,1000


gender,female,male,All
test preparation course,,,
completed,190,168,358
none,328,314,642
All,518,482,1000


## Another way of shuffling
How about creating a random array of 0s and 1s (0s for males, 1s for females)?<br>
you can create a random array of 0s and 1s using random.randint:<br>
https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.random.randint.html

### Now check the proportions

## 7. Calculate the stat
After you have created successfully shuffled the exchangeable variable in the dataset, you can calculate the stat.

In [86]:
ctab5 = pd.crosstab(df_copy['test preparation course'] == 'completed', df_copy['gender'], margins = True)
ctab5

gender,female,male,All
test preparation course,,,
False,328,314,642
True,190,168,358
All,518,482,1000


In [87]:
p_com_fem = ctab5['female'][True]/ctab5['All']['All']
p_com_m = ctab5['male'][True]/ctab5['All']['All']
print(p_com_fem)
print(p_com_m)

0.19
0.168


# All the steps above must be done in a loop (same as bootstrap)
like the bootstrap, you will have to perform the steps above within a loop with a specific number of iterations! You will end up with a stat for each iteration!<br>
Once the loop is done, you can calculate the p-value: 
* __One Sided:__ How many times the calculated stats were bigger than the actual value of the stat, calculated from the original data?/ number of times you have permuted the exchangeable variable.

### What you need to do to calculate the p-value?
The calculations you need to do will be similar to what we will do in this example:<br>
Suppose you have created an array of 1000 random integers between 0 and 100. What is the probability that created integers is higher than 60?

In [89]:
Ex_array = np.random.randint(101, size = 1000)
Ex_array

array([ 67,  76,  26,  36,   2,   4,  38,  96,  22,   5,  69,  73,   8,
        28,  10,  34,  46,  55,  28,  15,  61,  96,  41,  72,  68,  86,
        69,  33,  72,  34,  33,  40,  10,  37,   5,   4,   1,  45,  37,
        78,  60,  79,  16,   7,  93,   3,  64,  88,  94,  84,  65,  64,
        20,  73,  14,  33,  55,   1,   5,  33,  87,  69,  40,   2,  52,
        29,  65,  95,  53,  28,  13,  30,  13,  69,   5,  84,  63,  76,
         0,  83,  46,  74,  42,  15,  82,  12,  95,  45,  71,  93,  21,
        90,  39,  20,  76,  87,  79,  70,  82,  94,   8,  39,  21,  21,
        72,  96,  78,  19,  39,  72,  23,  32,  27,  15,  51,  67,  80,
        48,  85,  73,  59,  54,  54,  11,  87,  28,  11,  65,  20,  46,
        72,  95,  53,  33,  43,  84,  98,  91,  27,  51,  88,  14,   6,
        84,  32,  62,   5,  19,  97,  44,  77, 100,  15,   7,  92,  40,
        61,  65,  34,  81,  83,  70,  85,  39,  55,   3,  10,  46,  30,
        41,  24,  43,  65,  14,  69,  10,  87,  45,  54,   1,  9

In [ ]:
diff = Ex_array >= 60
diff

ntrues = sum(diff)
p_value = ntrues/len(diff)
p_value